# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

* Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
    * excel and vensim are only supported on windows
    * vensim requires 32 bit python, and a 7.1!! version of vensim DSS
    * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get it from [NetLogo 6.0](https://ccl.northwestern.edu/netlogo/download.shtml) 
    * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
    * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).
    

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes example scripts for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel). 

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments,
                           ema_logging, Constant)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis.plotting import lines, Density


def PredPrey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, final_time=365, reps=1):

    #Initial values
    predators, prey, sim_time = [np.zeros((reps, int(final_time/dt)+1)) for _ in range(3)]
    
    for r in range(reps):
        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

        #Calculate the time series
        for t in range(0, sim_time.shape[1]-1):

            dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
            dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

            prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
            predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
            sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}
print(PredPrey())

In [1]:
## excel

import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint
from ema_workbench import (RealParameter, TimeSeriesOutcome, ema_logging,
                           perform_experiments)

from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.em_framework.evaluators import MultiprocessingEvaluator, SequentialEvaluator
from ema_workbench.analysis import plotting, plotting_util

ema_logging.log_to_stderr(level=ema_logging.INFO)

model = ExcelModel("predatorPrey", wd='./model', model_file='./PredPrey.xlsx')

model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter("predator_loss_rate", 0.04, 0.08)]

model.outcomes =  [TimeSeriesOutcome('predators'), TimeSeriesOutcome('prey')]
# name of the sheet
model.default_sheet = "Sheet1"

with SequentialEvaluator(model) as evaluator:
    results = perform_experiments(model, 50, reporting_interval=1, evaluator=evaluator)

[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 6 cases completed
[MainProcess/INFO] 7 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 9 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 11 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] 13 cases completed
[MainProcess/INFO] 14 cases completed
[MainProcess/INFO] 15 cases completed
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 17 cases completed
[MainProcess/INFO] 18 cases completed
[MainProcess/INFO] 19 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 21 cases completed
[MainProcess/INFO] 22 cases completed
[MainProcess/INFO] 23 cases comp

In [12]:
## netlogo 

import numpy as np
import ctypes
import matplotlib.pyplot as plt

from scipy.integrate import odeint
from ema_workbench import (RealParameter, TimeSeriesOutcome, ema_logging,
                           perform_experiments)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.em_framework.evaluators import MultiprocessingEvaluator, SequentialEvaluator
from ema_workbench.analysis import plotting, plotting_util

ema_logging.log_to_stderr(level=ema_logging.INFO)

model = NetLogoModel("predatorPrey", wd='./model', model_file='./PredPrey.nlogo')
model.run_length = 365
model.replications = 1

model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter("predator_loss_rate", 0.04, 0.08)]

model.outcomes =  [TimeSeriesOutcome('predators'), TimeSeriesOutcome('prey')]

with SequentialEvaluator(model) as evaluator:
    results = perform_experiments(model, 50, reporting_interval=1, evaluator=evaluator)

[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/ERROR] No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.
Traceback (most recent call last):
  File "C:\Users\Martijn Ras\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 85, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\Martijn Ras\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ema_workbench\util\ema_logging.py", line 158, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\Martijn Ras\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ema_workbench\em_framework\model.py", line 296, in run_model
    super(Replicator, self).run_model(scenario, policy)
  File "C:\Users\Martijn Ras\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ema_workbench\util\ema_

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
## pysd

In [73]:
## not working python model....

import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint
from ema_workbench import Model, RealParameter, TimeSeriesOutcome, ScalarOutcome

model = Model('pypredprey', function=PredPrey)

model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter("predator_loss_rate", 0.04, 0.08)]

# model.levers = [RealParameter('initial_prey', 25, 75),
#                 RealParameter('initial_predators', 10,30)]

model.constants = [Constant('final_time', 365),
                    Constant('dt', 0.25)]

model.outcomes =  [TimeSeriesOutcome('predators'), TimeSeriesOutcome('prey')]

# run the experiments
from ema_workbench import (MultiprocessingEvaluator, SequentialEvaluator ,ema_logging,
                           perform_experiments)
ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50)
experiments, outcomes = results

import matplotlib.pyplot as plt # TO DO; plot the scalers.
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)
#print(experiments.shape)
#print(list(outcomes.keys()))

# policies = experiments['policy']
#print(policies)
#print("set policy")
# for i, policy in enumerate(np.unique(policies)):
#     experiments.loc[policies==policy, 'policy'] = str(i)
#print("changed policy")

# print(outcomes['predators'])
# for outcome in outcomes.keys():
#     for x in range(0, len(outcomes[outcome])):
#         for i in range(0, len(outcomes[outcome][x][0])):

# print(outcomes['predators'])
# data = pd.DataFrame(outcomes)
# #print("created df")
# data['policy'] = policies
#print("added policy to df")
# print(data)

# sns.pairplot(data, vars=list(outcomes.keys()), diag_kind='auto')
# #print("created plot")
# plt.show()
#print("done")
from ema_workbench.analysis import plotting, plotting_util

for outcome in outcomes.keys():
    print(outcome)
    plotting.lines(experiments, outcomes, outcomes_to_show=outcome, density=plotting_util.Density.HIST)
plt.show()


[MainProcess/INFO] performing 50 scenarios * 1 policies * 1 model(s) = 50 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 15 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 25 cases completed
[MainProcess/INFO] 30 cases completed
[MainProcess/INFO] 35 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 45 cases completed
[MainProcess/INFO] 50 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] no time dimension found in results
[[[ 20.          20.49783164  21.00790165 ...  48.64154076  49.63452807
    50.6278945 ]]

 [[ 20.          20.53110607  21.06893476 ...  14.20016508  14.0303137
    13.86235353]]

 [[ 20.          20.43436344  20.88097526 ...  87.50824776  89.33481735
    91.1676089 ]]

 ...

 [[ 20.          20.69124571  21.40867632 ... 109.39362702 108.24062624
   107.09868676]]

 [[ 20.          

AttributeError: 'NoneType' object has no attribute 'T'